In [2]:
# Import libraries
from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc

In [3]:
data = pd.read_csv('cov_merged.csv')
target = pd.read_csv('outcomes.csv')
print(data)
print(target)

            rowId  covariateId  covariateValue  \
0          3938.0   4083311102               1   
1        175397.0     81380102               1   
2         23031.0   4145418102               1   
3        152472.0    135473102               1   
4         60021.0   4213101102               1   
...           ...          ...             ...   
5411125   67932.0    378253104               1   
5411126  135007.0   4211852104               1   
5411127  127381.0    197988104               1   
5411128   20208.0    201826104               1   
5411129  161935.0    440005104               1   

                                             covariateName  analysisId  \
0        condition_occurrence during day -365 through -...         102   
1        condition_occurrence during day -365 through -...         102   
2        condition_occurrence during day -365 through -...         102   
3        condition_occurrence during day -365 through -...         102   
4        condition_occurrence

In [27]:
# Create windows to represent analysis ids
analysisId_to_window = {101: "all", 102: '365d', 103: '180d', 104: '030d', 105: 'all',
                        106: '365d', 107: '180d', 108: '030d'}

# Create observations data file from data file
observation = pd.DataFrame()
observation['conceptId'] = data['conceptId']
# observation['analysisId'] = data['analysisId']
observation['window'] = data['analysisId'].apply(lambda x: analysisId_to_window.get(x, 'Unknown'))
observation['covariateValue'] = 1
observation['patient'] = data['rowId'].astype(int)
# observation['covariateId'] = data['covariateId']
print(observation)
print(observation.drop_duplicates())
observation = observation.drop_duplicates()
print(len(set(observation['patient'])))

         conceptId window  covariateValue  patient
0          4083311   365d               1     3938
1            81380   365d               1   175397
2          4145418   365d               1    23031
3           135473   365d               1   152472
4          4213101   365d               1    60021
...            ...    ...             ...      ...
5411125     378253   030d               1    67932
5411126    4211852   030d               1   135007
5411127     197988   030d               1   127381
5411128     201826   030d               1    20208
5411129     440005   030d               1   161935

[5411130 rows x 4 columns]
         conceptId window  covariateValue  patient
0          4083311   365d               1     3938
1            81380   365d               1   175397
2          4145418   365d               1    23031
3           135473   365d               1   152472
4          4213101   365d               1    60021
...            ...    ...             ...      ...
541

In [12]:
observation_all = observation[observation['window'] == "all"]
print(len(observation_all))
observation_365d = observation[observation['window'] == "365d"]
print(len(observation_365d))
observation_180d = observation[observation['window'] == "180d"]
print(len(observation_180d))
observation_030d = observation[observation['window'] == "030d"]
print(len(observation_030d))

# Pick time window
chosen_window = "365d"
observation = observation[observation['window'] == chosen_window]

4200550
650535
382161
110648


In [13]:
risk_window_end = 1825
# Creating outcomes list
outcomes = []
has_outcome = 0
no_outcome = 0
for i in observation['patient']:
    if i in target['rowId'].values:
        if target.loc[target['rowId'] == i, 'daysToEvent'].values[0] < risk_window_end:
            outcomes.append(1)
            has_outcome += 1
        else:
            outcomes.append(0)
            no_outcome += 1
    else:
        outcomes.append(0)
        no_outcome += 1

print(f'Has outcome {has_outcome}')
print(f'No outcome {no_outcome}')

Has outcome 1172
No outcome 649363


In [14]:
random_state = 42
test_size = 0.25
# Split the data in train and test set
X_train, X_test, y_train, y_test = train_test_split(observation, outcomes, stratify=outcomes, random_state=random_state, test_size=test_size)

In [15]:
# Load ancestor and outcomes data
ancestry = pd.read_csv("relations_ancestor_total.csv")

In [16]:
# Choose what level above the data stays
chosen_level = 4
# Filter relations above chosen level
filtered_relations = ancestry[
    (ancestry['ANCESTOR_CON CEPT_ID'] == 441840) &
    (ancestry['MIN_LEVELS_OF_SEPARATION'] < chosen_level)
]

print(len(set(filtered_relations['DESCENDANT_CONCEPT_ID'])))

17829


In [17]:
# Select only concepts with observed descendants
selected_concepts_set = set(filtered_relations['DESCENDANT_CONCEPT_ID'])
observed_concepts_set = set(X_train['conceptId'])
observed_descendants = ancestry[ancestry['DESCENDANT_CONCEPT_ID'].isin(observed_concepts_set)]
concepts_with_observed_descendants_set = set(observed_descendants['ANCESTOR_CONCEPT_ID'])

In [18]:
# Count how many times each concept appears as an ancestor of observed descendants
concept_to_score = observed_descendants['ANCESTOR_CONCEPT_ID'].value_counts().to_dict()

In [19]:
# Function to compute cumulative score for each row
def compute_cumulative_score(concept_id):
    score = 0
    if concept_id in selected_concepts_set:
        score += 1
        score += concept_to_score.get(concept_id, 0)
    return score

# Apply to training and test sets
X_train['covariateValue'] = X_train['conceptId'].apply(compute_cumulative_score)
print(X_train)
print(X_train[X_train['covariateValue'] > 0])

        conceptId window  covariateValue  patient
126958     442774   365d               0    23202
558864    4182210   365d              13    44258
194507     194133   365d               2   123142
308071     199876   365d               0    92767
529225    4088777   365d               0   148512
...           ...    ...             ...      ...
121323     444187   365d               4   128897
380138    4185503   365d              87   128113
43502    40481632   365d               4    94895
363649     137351   365d               0    23226
449576    4022201   365d              36   163841

[487901 rows x 4 columns]
        conceptId window  covariateValue  patient
558864    4182210   365d              13    44258
194507     194133   365d               2   123142
337596     435613   365d               3   125694
1694       440921   365d              35    46806
275433    4304008   365d               3   126914
...           ...    ...             ...      ...
390360     444220   365

In [20]:
def pivot_covariates(df):
    observation = df.pivot(index='patient', columns='conceptId', values='covariateValue')
    return observation.fillna(0)

X_train = pivot_covariates(X_train)
X_test = pivot_covariates(X_test)

print(len(X_train))
print(len(X_test))
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

144301
92838


In [21]:
print(len(X_train))
print(len(X_test))
print(len(y_train))

144301
92838
487901


In [22]:
def create_outcomes(indexes):
    outcomes = []
    for i in indexes:
        if i in target['rowId'].values:
            days_to_event = target.loc[target['rowId'] == i, 'daysToEvent'].values[0]
            outcomes.append(1 if days_to_event < risk_window_end else 0)
        else:
            outcomes.append(0)
    return outcomes

y_train = create_outcomes(X_train.index)
y_test = create_outcomes(X_test.index)

In [23]:
lr_penalty = 'l1'
lr_solver = "liblinear"
model = LogisticRegression(penalty=lr_penalty, solver=lr_solver)

In [24]:
# Fit model on train data
model.fit(X_train, y_train)


# Get model coefficients
model_coefficients = model.coef_[0]
print(model.coef_)

number_model_features = len(set(model_coefficients))

print('Number of model features:', number_model_features)

# Get train probabilities
y_prob_train = model.predict_proba(X_train)

# Get ths train auc score
train_auc = roc_auc_score(y_train, y_prob_train[:, 1])
print(train_auc)

# Get test probabilities
y_prob_test = model.predict_proba(X_test)

# Get ths test auc score
test_auc = roc_auc_score(y_test, y_prob_test[:, 1])
print(test_auc)

# Calculate precision-recall curve
precision_test, recall_test, _ = precision_recall_curve(y_test, y_prob_test[:, 1])

# Calculate AUPRC
auprc_test_score = auc(recall_test, precision_test)

[[0.         0.         0.         ... 0.         0.         0.65285581]]
Number of model features: 142
0.6892663545980868
0.4996038681742332


In [25]:
import numpy as np
from sklearn.metrics import accuracy_score

# Match the positive rate in training (more realistic) or use 0.5 for fully random
positive_rate = np.mean(y_train)
random_preds = np.random.choice([0, 1], size=len(y_test), p=[1 - positive_rate, positive_rate])

# Compute accuracy of random model
random_model_accuracy = accuracy_score(y_test, random_preds)
print(random_model_accuracy)

0.9967577931450483
